In [1]:
from qiskit import QuantumCircuit, execute
from qiskit.circuit import ParameterVector
from qiskit.circuit.random import random_circuit
import random
from qiskit import Aer
from qiskit.aqua.components.optimizers import ADAM
import numpy as np

/home/luis/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Prepare 4 random 4-qubit quantum states of your choice.
Create and train a variational circuit that transforms input states into predefined output states. Namely
if random state 1 is provided, it returns state |0011>
if random state 2 is provided, it returns state |0101>
if random state 3 is provided, it returns state |1010>
if random state 4 is provided, it returns state |1100>
What would happen if you provided a different state?

Analyze and discuss the results.

In [44]:
def generate_random_circuits(seed1,seed2,seed3,seed4):
    qc1 = random_circuit(4, 2, max_operands=3, seed=seed1)
    qc2 = random_circuit(4, 2, max_operands=3, seed=seed2)
    qc3 = random_circuit(4, 2, max_operands=3, seed=seed3)
    qc4 = random_circuit(4, 2, max_operands=3, seed=seed4)
    return [qc1,qc2,qc3,qc4]


def get_prob_distribution(circuit,backend, params):
    #Assign parameters using the assign_parameters method
    bound_circuit = circuit.bind_parameters(params)
    bound_circuit.measure_all()
    raw_distrubution = execute(bound_circuit,backend,shots=1000).result().get_counts()
    for key in raw_distrubution.keys():
        raw_distrubution[key] = raw_distrubution[key]/1000
    return raw_distrubution

def cost_function(expected_val,distribution,epsilon=0.0005):
    value = 0
    if(expected_val in distribution):
        value += np.log(max(epsilon,distribution[expected_val]))
    else:
        value += np.log(epsilon)            
    return (-1/len(expected_val))*(value)
        


def objective_function(circuit, label, params):
    n = 4
    num_layers = 2
    circuit.append(generate_anzats(n,num_layers),list(range(0,n)))
    distr = get_prob_distribution(circuit,Aer.get_backend('qasm_simulator'), params )
    return cost_function(label, distr)

def train(circuit, label, init_params):
    optimizer = ADAM(maxiter=100, )
    objective_function = lambda params: objective_function(circuit, label, params)
    #opt_params, value, _ = optimizer.optimize(len(init_params), objective_function, initial_point=init_params)
    return objective_function(circuit, label, init_params)

def generate_anzats(n_qubits,n_layers,type_r=None):

    #ParameterVectors are initialized with a string identifier and an integer specifying the vector length
    parameters = ParameterVector('θ', (n_qubits*(n_layers+1)))
    circuit = QuantumCircuit(n_qubits)

    for layer in range(n_layers):
        for i in range(n_qubits):
            if(type_r == "qgan"):
                circuit.ry(parameters[n_qubits*layer+i], i)
            else:
                circuit.rx(parameters[n_qubits*layer+i], i)

        #Appending the entangling Controlled gates
        for i in range(n_qubits):
            if(type_r == "qgan"):
                circuit.cz(i,(i+1)%n_qubits)
            else:
                for j in range(i+1,n_qubits):
                    circuit.cx(i,j)

    #Appending one additional layer of parameterized Ry gates
    for i in range(n_qubits):
        if(type_r == "qgan"):
            circuit.ry(parameters[n_qubits*(n_layers)+i], i)
        else:
            circuit.rx(parameters[n_qubits*(n_layers)+i], i)
    circuit.barrier()
    return circuit


In [45]:
seed1 = 1
seed2 = 2
seed3 = 3
seed4 = 4
circuits = generate_random_circuits(seed1,seed2,seed3,seed4)
labels = ["0011","0101", "1010", "1100"]
circuit = circuits[0]
n = 4
num_layers = 2
param_list = [random.randint(0,20) for x in range(0,(n*(num_layers+1)))]
objective_function(circuit, labels[0], param_list)
train(circuit, "0011", param_list)


TypeError: <lambda>() takes 1 positional argument but 3 were given